In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

### Código para calcular la distancia geométrica entre un pixel y uno de sus vecinos.

In [2]:
def geoDist(x,y):
    
    (s,t) = x
    (m,n) = y
    
    numD = np.power(((m+n)/2 - (s+t)/2),2) # DUDA IMPORTANTE
    denD = 2*np.power(10,2)
    
    return math.exp(((-1)*numD)/denD)

### Código para calcular la diferencia de intensidad entre un pixel y uno de sus vecinos.

In [3]:
def intDif(gx,gy):
    
    numI = np.power((gx-gy),2)
    denI = 2*np.power(6.7,2)
    
    return math.exp(((-1)*numI)/denI)

### Código para calcular la similitud entre un pixel y uno de sus vecinos.

In [6]:
def simil(img,x,y):
    
    (s,t) = x
    (m,n) = y
    
    gx = img[s,t]
    gy = img[m,n]
    
    return geoDist(x,y)*intDif(gx,gy)

### Código para calcular la similitud local de cada pixel en una matriz.

In [22]:
def imgSimil(img):
    
    imgS = img.copy()
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 1 and s < img.shape[0]-2 and t > 1 and t < img.shape[1]-2:
                nx = list()
                for i in range(-2,3):
                    for j in range(-2,3):
                        if not i==j==0:
                            nx.append((s+i,t+j))
                dsetaX = 0
                for y in nx:
                    dsetaX += simil(img,x,y)
                imgS[s,t] = dsetaX
            else:
                imgS[s,t] = 0
    
    return imgS

In [24]:
def imgLocalSimil(img, imgS):
    
    imgLS = img.copy()
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
                nx = list()
                for i in range(-2,3):
                    for j in range(-2,3):
                        if not i==j==0:
                            nx.append((s+i,t+j))
                dsetaY = 0
                for y in nx:
                    (m,n) = y
                    dsetaY += imgS[m,n]
                dsetaX = imgS[s,t]
                medDsetaX = dsetaX / (dsetaY/24)
                if medDsetaX > 2.5:
                    imgLS[s,t] = 1
                else:
                    imgLS[s,t] = medDsetaX/2.5
            else:
                imgLS[s,t] = 0
    
    return imgLS

### Código para calcular la región designada de cada pixel.

In [2]:
def pixReg(img,imgLS):
    
    w1 = 2
    w2 = 4
    tSigma = 0.12
    
    flatReg = list()
    compReg = list()
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
                nx = list()
                for i in range(-2,3):
                    for j in range(-2,3):
                        if not i==j==0:
                            nx.append((s+i,t+j))
                a = 0
                b = 0
                for y in nx:
                    (m,n) = y
                    a += np.power(imgLS[m,n],w1)
                    b += np.power(imgLS[m,n],w2)
                miX = 0
                for y in nx:
                    (m,n) = y
                    miX = (np.power(imgLS[m,n],w1)*img[m,n])/a
                sigmaX = 0
                for y in nx:
                    (m,n) = y
                    sigmaX = (np.power(imgLS[m,n],w2)*np.power(img[m,n]-miX,2))/b
    
                if sigmaX <= tSigma:
                    flatReg.append((s,t))
                else:
                    compReg.append((s,t))
    
    return (flatReg,compReg)

### Código para calcular la naturaleza de un pixel.

In [4]:
def pixNat(img,imgLS):
    
    noise = list()
    clean = list()
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
                nx = list()
                for i in range(-2,3):
                    for j in range(-2,3):
                        if not i==j==0:
                            nx.append((s+i,t+j))
                iX = img[s,t]
                yLS = list()
                for y in nx:
                    (m,n) = y
                    yLS.append(imgLS[m,n])
                iY = max(yLS)
                
                if iX - iY > 5:
                    noise.append((s,t))
                else:
                    clean.append((s,t))
    
    return (noise,clean)

### Código para calcular el nivel de ruido en la zona.

In [5]:
def noiseLvl(img,regiones,naturaleza):
    
    (flatReg,compReg) = regiones
    (noise,clean) = naturaleza
    
    for x in range(10):
        (s,t) = flatReg[x]
        nx = list()
        for i in range(-2,3):
            for j in range(-2,3):
                if not i==j==0:
                    nx.append((s+i,t+j))
        if flatReg[x] in noise:
            n = 1
            c = 0
        elif flatReg[x] in clean:
            n = 0
            c = 1
        for y in nx:
            if y in noise:
                n += 1
            elif y in clean:
                c += 1
        sigma += n / (c+n)
        
    sigma = sigma/10
    
    thetaF = (-0.12)*np.power(sigma,3) + 0.07*np.power(sigma,2) + 0.75*sigma + 0.19
    thetaC = 0.31*np.power(sigma,3) + 0.63*np.power(sigma,2) + 0.52*sigma + 0.03
    
    
    return (thetaF,thetaC)

### Código para catalogar de forma final los píxeles de la imagen.

In [5]:
def filtrado(img):
    imgS = imgSimil(img)
    imgLS = imgLocalSimil(img,imgS)
    regiones = pixReg(img,imgLS)
    naturaleza = pixNat(img,imgLS)
    (thetaF,thetaC) = noiseLvl(img,regiones,naturaleza)
    (flatR,compR) = regiones
    pixelesR = list()
    pixelesL = list()

    for p in flatR:
        (s,t) = p
        lsp = imgLS[s,t]
        if lsp <= thetaF:
            pixelesR.append(p)
        else:
            pixelesL.append(p)
    
    for p in compR:
        (s,t) = p
        lsp = imgLS[s,t]
        if lsp <= thetaC:
            pixelesR.append(p)
        else:
            pixelesL.append(p)
    
    medF = cv.medianBlur(img)
    gaussF = cv.GaussianBlur(medF)

    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if (s == 0 or t == 0 or s == img.shape[0]-1 or t == img.shape[1]-1) and ((s,t) in pixelesR):
                valor = img[s,t] - gaussF[s,t]
                valorAb = abs(valor)
                if valorAb <= 15:
                    pixelesL.append(s,t)
                    pixelesR.remove(s,t)
    
    return (pixelesR,pixelesL)

### Código para calcular el valor de intensidad de un pixel ruidoso filtrado.

In [13]:
def intVal(img,x):
    
   

### Código para calcular el PSNR de dos imagenes.

In [14]:
def PSNR(imgO,imgR):
    
    mse = np.mean(np.power(imgO-imgR,2))
    if mse == 0:
        return 100
    
    psnr = 10*np.log10(np.power(255,2)/mse)
    return psnr

### Código para introducir RVIN en imagenes.

In [ ]:
cv.im

### Código de limpieza de imagenes con RVIN.

In [21]:
not (0 <= 1 and 1<=1)

False